In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

### Import CSV and Perform Data Cleaning

In [4]:
columns = ["ID", "title", "study_type", "arm_group_type", "intervention_type",
          "interventional_type, model", "phase", "fda_regulated_drug", "fda_regulated_device"]

target = ["status"]

In [6]:
# Load the interventional_study_only
file_path = Path('Tables/interventional_study_only.csv')
df = pd.read_csv(file_path)
df

,ID,title,status,study_type,arm_group_type,intervention_type,interventional_type_model,phase,fda_regulated_drug,fda_regulated_device
0,IRST174.22,What is the Best Interval to Screen Women 45-4...,Recruiting,Interventional,"No Intervention, Experimental, Experimental",Diagnostic Test,Parallel Assignment,Not Applicable,No,No
1,GCO 17-2188,Increasing African Immigrant Womens Participat...,Recruiting,Interventional,Other,Behavioral,Single Group Assignment,Not Applicable,No,No
2,233756,The HIFUB Study (HIFU in Breast Cancer),Not yet recruiting,Interventional,Experimental,Device,Single Group Assignment,Not Applicable,No,No
3,D16196,A Pilot Multi-Institutional Study to Evaluate ...,Completed,Interventional,Experimental,Device,Single Group Assignment,Not Applicable,No,Yes
4,RO1912-30902,RCT Comparing 2 Radiotherapy HypoFractionation...,Unknown status,Interventional,"Other, Experimental",Radiation,Parallel Assignment,Not Applicable,No,No
...,...,...,...,...,...,...,...,...,...,...
3913,20201491,Culturally Tailored Nurse Coaching Study for C...,Recruiting,Interventional,"Experimental, Active Comparator","Behavioral, Behavioral",Parallel Assignment,Not Applicable,No,No
3914,32900654326,"TPVB, PECSB, ESPB for Postmastectmy Pain",Completed,Interventional,"Placebo Comparator, Active Comparator, Active ...","Procedure, Drug",Parallel Assignment,Not Applicable,No,No
3915,ReDA 13176,A Randomized Phase III Trial of Stereotactic A...,Not yet recruiting,Interventional,"Active Comparator, Experimental","Radiation, Drug, Drug, Drug, Drug, Other, Radi...",Parallel Assignment,Phase 3,No,No
3916,64121317.4.1001.5330,Nivolumab in Prostate Cancer With DNA Repair D...,"Active, not recruiting",Interventional,Experimental,Drug,Single Group Assignment,Phase 2,Yes,No


In [ ]:
# Examine DataFrame

# Extract and keep first values for arm_group_type and intervention_type

# Drop null row
#.dropna()

# Remove "Unknown" ("recruiting", "not yet recruiting", "active, not recruiting", "enrolling by invitation"?) status
unknow_stat = df['status'] != 'Unknown status'
df = df.loc[unknown_stat]

# Convert the target Column values to low_chance and high_chance based in their values
x = {'Completed' : 'high_chance'}
df = df.replace(x)

x - dict.fromkeys (['Withdrawn', 'Terminated', 'Suspended'], 'low_chance')
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

### Split the Data into Training and Testing

In [ ]:
# Create our features
X = pd.get_dummies (df, columns=['arm_group_type', 'intervention_type', 'intervenal_type_model',
                                'phase', 'fda_regulated_drug', 'fda_regulated_device']).drop('status', axis=1)

# Create our target
y = df['status']
X.head()

In [ ]:
# Check count, mean, max for features
X.describe()

In [ ]:
# Check the balance of our target values
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
print(Counter(y_train))
print(Counter(y_test))

### Balanced Random Forest Classifier

In [ ]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

# Instantiate model
classifier = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

# Fit model
classifier.fit(X_train, y_train)

In [ ]:
# Calculated the balanced accuracy score
y_pred = classifier.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual low_chance", "Actual high_chance"], columns=["Predicted low_chance", "Predicted high_chance"])
cm_df

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# List the features sorted in descending order by feature importance
features_rank = sorted(zip(classifier.feature_importances_, X.columns), reverse=True)
for feature in features_rank:
    print(f"{feature[1]}: ({feature[0]})")

### Optimize model, Use resampling techniques to determine the best performing algorithm

#### Oversampling

#### Undersampling

#### Combination (Over and Under) Sampling